In [ ]:
import joblib

In [ ]:
#loaded_svm = joblib.load('svm_classifier.pkl')

# Cargar el modelo
model = load_model('model_resize_15-15.h5')

# Cargar la lista de historias
with open('histories_resize_15-15.pkl', 'rb') as file:
    history_list = pickle.load(file)

In [ ]:
train_acc = model_history.history['accuracy']
val_acc = model_history.history['val_accuracy']

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Crear una figura y ejes para el gráfico
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 6))

history_acc = []
history_val_acc = []
history_loss = []
history_val_loss = []

# Recopilar las precisiones (accuracy) de todas las historias en history_list
for history in history_list:
    history_acc.extend(history['acc'])
    history_val_acc.extend(history['val_acc'])
    history_loss.extend(history['loss'])
    history_val_loss.extend(history['val_loss'])

# Graficar la precisión (accuracy) en función de las épocas
axes[0].plot(history_acc, label='Train Accuracy')
axes[0].plot(history_val_acc, label='Validation Accuracy')
axes[0].set_title('Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()

axes[1].plot(history_loss, label='Train Loss')
axes[1].plot(history_val_loss, label='Validation Loss')
axes[1].set_title('Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Labels des axes
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

# Courbe de la précision sur l'échantillon d'entrainement
plt.plot(np.arange(1 , epochs + 1, 1),
         model_history.history['accuracy'],
         label = 'Training Accuracy',
         color = 'blue')

# Courbe de la précision sur l'échantillon de test
plt.plot(np.arange(1 , epochs + 1, 1),
         model_history.history['val_accuracy'], 
         label = 'Validation Accuracy',
         color = 'red')

# Affichage de la légende
plt.legend()

# Affichage de la figure
plt.show()